# Import Libraries:  

In [2]:
import pandas as pd 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import io
from sqlalchemy import create_engine



# Crawl website:

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get('https://coinmarketcap.com/currencies/bitcoin/historical-data/')


for i in range (2):
    button = driver.find_element(By.CSS_SELECTOR,"#__next > div > div.main-content > div.sc-4vztjb-0.cLXodu.cmc-body-wrapper > div > div.sc-1prm8qw-0.bUVslT.container > div > div > div.history > p:nth-child(3) > button")
    driver.execute_script("arguments[0].click();", button)  

col_names = driver.find_elements(By.CSS_SELECTOR, ".stickyTop")
col_names_text = [i.text for i in col_names]

rest_cols = []
for i in range(1,8):
    rest_col_list = driver.find_elements(By.CSS_SELECTOR, f"td:nth-child({i})") 
    col_list_text = [i.text for i in rest_col_list] 
    rest_cols.append(col_list_text)



# Creating function to store in database

In [4]:
def store_data(username, password, host, port, db_name, df,sql_table_name):

        engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{db_name}')

        df.head(0).to_sql(sql_table_name, engine, if_exists='replace',index=False) #drops old table and creates new empty table

        conn = engine.raw_connection()
        cur = conn.cursor()
        output = io.StringIO()
        df.to_csv(output, sep='\t', header=False, index=False)
        output.seek(0)
        contents = output.getvalue()
        cur.copy_from(output, sql_table_name, null="") # null values become ''
        conn.commit()


# Storing in Database:

In [5]:
df = pd.DataFrame(rest_cols ).T
df.columns = col_names_text


store_data('postgres', '1234', 'localhost', '5432', 'coinmarketcap', df, 'btc2')